## এম্বেডিংস

আমাদের আগের উদাহরণে, আমরা `vocab_size` দৈর্ঘ্যের উচ্চ-মাত্রার ব্যাগ-অফ-ওয়ার্ডস ভেক্টরের উপর কাজ করেছি এবং নিম্ন-মাত্রার পজিশনাল রিপ্রেজেন্টেশন ভেক্টরগুলোকে স্পষ্টভাবে স্পার্স ওয়ান-হট রিপ্রেজেন্টেশনে রূপান্তর করেছি। এই ওয়ান-হট রিপ্রেজেন্টেশন মেমোরি-সাশ্রয়ী নয়। এছাড়াও, প্রতিটি শব্দকে একে অপরের থেকে স্বাধীনভাবে বিবেচনা করা হয়, তাই ওয়ান-হট এনকোডেড ভেক্টরগুলো শব্দগুলোর মধ্যে অর্থগত সাদৃশ্য প্রকাশ করতে পারে না।

এই ইউনিটে, আমরা **News AG** ডেটাসেট নিয়ে আরও অনুসন্ধান চালিয়ে যাব। শুরু করার জন্য, চলুন ডেটা লোড করি এবং আগের ইউনিট থেকে কিছু সংজ্ঞা পুনরায় দেখি।


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### এম্বেডিং কী?

**এম্বেডিং** ধারণাটি হলো শব্দগুলোকে নিম্ন-মাত্রার ঘন ভেক্টর দিয়ে উপস্থাপন করা, যা শব্দটির অর্থবহ ধারণা প্রতিফলিত করে। আমরা পরে আলোচনা করব কীভাবে অর্থবহ শব্দ এম্বেডিং তৈরি করা যায়, তবে আপাতত এম্বেডিংকে একটি পদ্ধতি হিসেবে ভাবুন যা শব্দ ভেক্টরের মাত্রা কমিয়ে আনে।

সুতরাং, একটি এম্বেডিং লেয়ার একটি শব্দকে ইনপুট হিসেবে নেয় এবং নির্দিষ্ট `embedding_size` আউটপুট ভেক্টর প্রদান করে। এক অর্থে, এটি একটি `Dense` লেয়ারের মতোই, তবে এটি এক-হট এনকোডেড ভেক্টর ইনপুট হিসেবে নেওয়ার পরিবর্তে একটি শব্দ নম্বর নিতে সক্ষম।

আমাদের নেটওয়ার্কে প্রথম লেয়ার হিসেবে একটি এম্বেডিং লেয়ার ব্যবহার করে, আমরা ব্যাগ-অফ-ওয়ার্ডস থেকে **এম্বেডিং ব্যাগ** মডেলে স্যুইচ করতে পারি, যেখানে আমরা প্রথমে আমাদের টেক্সটের প্রতিটি শব্দকে সংশ্লিষ্ট এম্বেডিং-এ রূপান্তর করি এবং তারপর সেই এম্বেডিংগুলোর উপর একটি সমষ্টিগত ফাংশন (যেমন `sum`, `average` বা `max`) গণনা করি।

![পাঁচটি সিকোয়েন্স শব্দের জন্য একটি এম্বেডিং ক্লাসিফায়ার দেখানো চিত্র।](../../../../../translated_images/bn/embedding-classifier-example.b77f021a7ee67eee.webp)

আমাদের ক্লাসিফায়ার নিউরাল নেটওয়ার্কে নিম্নলিখিত লেয়ারগুলো অন্তর্ভুক্ত রয়েছে:

* `TextVectorization` লেয়ার, যা একটি স্ট্রিং ইনপুট হিসেবে নেয় এবং টোকেন নম্বরের একটি টেনসর আউটপুট দেয়। আমরা একটি যুক্তিসঙ্গত শব্দভাণ্ডারের আকার `vocab_size` নির্ধারণ করব এবং কম ব্যবহৃত শব্দগুলো উপেক্ষা করব। ইনপুট শেপ হবে ১, এবং আউটপুট শেপ হবে $n$, কারণ আমরা $n$ টোকেন পাব, যার প্রতিটিতে ০ থেকে `vocab_size` পর্যন্ত নম্বর থাকবে।
* `Embedding` লেয়ার, যা $n$ নম্বর নেয় এবং প্রতিটি নম্বরকে একটি নির্দিষ্ট দৈর্ঘ্যের ঘন ভেক্টরে রূপান্তর করে (আমাদের উদাহরণে ১০০)। সুতরাং, $n$ শেপের ইনপুট টেনসর $n\times 100$ শেপের টেনসরে রূপান্তরিত হবে।
* অ্যাগ্রিগেশন লেয়ার, যা এই টেনসরের প্রথম অক্ষ বরাবর গড় গণনা করে, অর্থাৎ এটি বিভিন্ন শব্দের সাথে সম্পর্কিত $n$ ইনপুট টেনসরের গড় গণনা করবে। এই লেয়ারটি বাস্তবায়নের জন্য, আমরা একটি `Lambda` লেয়ার ব্যবহার করব এবং এতে গড় গণনার ফাংশন পাস করব। আউটপুটের শেপ হবে ১০০, এবং এটি পুরো ইনপুট সিকোয়েন্সের একটি সংখ্যাগত উপস্থাপনা হবে।
* চূড়ান্ত `Dense` লিনিয়ার ক্লাসিফায়ার।


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` প্রিন্টআউট-এ, **output shape** কলামে প্রথম টেনসরের ডাইমেনশন `None` মিনিব্যাচের আকার নির্দেশ করে, এবং দ্বিতীয়টি টোকেন সিকোয়েন্সের দৈর্ঘ্য নির্দেশ করে। মিনিব্যাচের সব টোকেন সিকোয়েন্সের দৈর্ঘ্য ভিন্ন হয়। আমরা পরবর্তী অংশে এটি কীভাবে সামলানো যায় তা নিয়ে আলোচনা করব।

এখন চলুন নেটওয়ার্কটি প্রশিক্ষণ দেই:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **নোট** যে আমরা ডেটার একটি উপসেটের উপর ভিত্তি করে ভেক্টরাইজার তৈরি করছি। এটি প্রক্রিয়াটি দ্রুত করার জন্য করা হয়, এবং এটি এমন একটি পরিস্থিতি তৈরি করতে পারে যেখানে আমাদের টেক্সটের সমস্ত টোকেন শব্দভাণ্ডারে উপস্থিত না থাকে। এই ক্ষেত্রে, সেই টোকেনগুলি উপেক্ষা করা হবে, যা সামান্য কম সঠিকতা সৃষ্টি করতে পারে। তবে, বাস্তব জীবনে টেক্সটের একটি উপসেট প্রায়শই একটি ভাল শব্দভাণ্ডার অনুমান প্রদান করে।


### ভেরিয়েবল সিকোয়েন্স সাইজ নিয়ে কাজ করা

চলুন বুঝি কীভাবে মিনি-ব্যাচে প্রশিক্ষণ হয়। উপরের উদাহরণে, ইনপুট টেনসরের মাত্রা 1, এবং আমরা ১২৮-লম্বা মিনি-ব্যাচ ব্যবহার করি, তাই টেনসরের প্রকৃত আকার হয় $128 \times 1$। তবে, প্রতিটি বাক্যের টোকেনের সংখ্যা ভিন্ন। যদি আমরা একটি একক ইনপুটে `TextVectorization` লেয়ার প্রয়োগ করি, তাহলে টোকেনাইজেশনের উপর নির্ভর করে ফেরত আসা টোকেনের সংখ্যা ভিন্ন হয়:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


তবে, যখন আমরা ভেক্টরাইজারকে একাধিক সিকোয়েন্সে প্রয়োগ করি, এটি আয়তাকার আকৃতির একটি টেনসর তৈরি করতে হয়, তাই এটি অব্যবহৃত উপাদানগুলোকে PAD টোকেন (যা আমাদের ক্ষেত্রে শূন্য) দিয়ে পূরণ করে:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

এখানে আমরা এম্বেডিংগুলি দেখতে পারি:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **নোট**: প্যাডিংয়ের পরিমাণ কমানোর জন্য, কিছু ক্ষেত্রে ডেটাসেটের সমস্ত সিকোয়েন্সকে দৈর্ঘ্য বৃদ্ধির ক্রমে (অথবা, আরও সঠিকভাবে, টোকেনের সংখ্যার ক্রমে) সাজানো যুক্তিসঙ্গত হতে পারে। এটি নিশ্চিত করবে যে প্রতিটি মিনিব্যাচে একই দৈর্ঘ্যের সিকোয়েন্স থাকবে।


## সেমান্টিক এম্বেডিংস: ওয়ার্ড২ভেক

আমাদের আগের উদাহরণে, এম্বেডিং লেয়ার শব্দগুলোকে ভেক্টর রূপে মানচিত্রিত করতে শিখেছিল, তবে এই ভেক্টরগুলোর সেমান্টিক অর্থ ছিল না। যদি এমন একটি ভেক্টর উপস্থাপনা শেখা যেত যেখানে একই ধরনের শব্দ বা সমার্থক শব্দগুলো এমন ভেক্টরের সাথে মিলে যেত যা কিছু ভেক্টর দূরত্ব (যেমন ইউক্লিডিয়ান দূরত্ব) অনুযায়ী কাছাকাছি থাকে, তাহলে ভালো হতো।

এটি করতে, আমাদের একটি বড় টেক্সট সংগ্রহে আমাদের এম্বেডিং মডেলকে প্রি-ট্রেইন করতে হবে, যেমন [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) পদ্ধতি ব্যবহার করে। এটি দুটি প্রধান আর্কিটেকচারের উপর ভিত্তি করে তৈরি, যা শব্দগুলোর একটি বিতরণকৃত উপস্থাপনা তৈরি করতে ব্যবহৃত হয়:

 - **কন্টিনিউয়াস ব্যাগ-অফ-ওয়ার্ডস** (CBoW), যেখানে আমরা মডেলটিকে পারিপার্শ্বিক প্রসঙ্গ থেকে একটি শব্দ অনুমান করতে প্রশিক্ষণ দিই। যদি ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ দেওয়া হয়, মডেলের লক্ষ্য হলো $(W_{-2},W_{-1},W_1,W_2)$ থেকে $W_0$ অনুমান করা।
 - **কন্টিনিউয়াস স্কিপ-গ্রাম** CBoW-এর বিপরীত। এই মডেলটি বর্তমান শব্দ অনুমান করতে পারিপার্শ্বিক প্রসঙ্গের উইন্ডো ব্যবহার করে।

CBoW দ্রুততর, এবং স্কিপ-গ্রাম ধীর হলেও এটি কম ঘন ঘন ব্যবহৃত শব্দগুলোর উপস্থাপনা আরও ভালোভাবে করে।

![CBoW এবং স্কিপ-গ্রাম অ্যালগরিদমের মাধ্যমে শব্দগুলোকে ভেক্টরে রূপান্তর করার একটি চিত্র।](../../../../../translated_images/bn/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

গুগল নিউজ ডেটাসেটে প্রি-ট্রেইন করা Word2Vec এম্বেডিং নিয়ে পরীক্ষা করার জন্য, আমরা **gensim** লাইব্রেরি ব্যবহার করতে পারি। নিচে আমরা 'neural' শব্দটির সাথে সবচেয়ে সাদৃশ্যপূর্ণ শব্দগুলো খুঁজে বের করব।

> **Note:** যখন আপনি প্রথমবার শব্দ ভেক্টর তৈরি করবেন, তখন সেগুলো ডাউনলোড করতে কিছুটা সময় লাগতে পারে!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


আমরা শব্দ থেকে ভেক্টর এমবেডিংও বের করতে পারি, যা শ্রেণীবিন্যাস মডেল প্রশিক্ষণে ব্যবহার করা হবে। এমবেডিং-এর ৩০০টি উপাদান রয়েছে, তবে এখানে স্পষ্টতার জন্য আমরা শুধুমাত্র ভেক্টরের প্রথম ২০টি উপাদান দেখাচ্ছি:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

সেমান্টিক এম্বেডিংসের দুর্দান্ত বিষয় হল আপনি সেমান্টিক্সের উপর ভিত্তি করে ভেক্টর এনকোডিং পরিচালনা করতে পারেন। উদাহরণস্বরূপ, আমরা এমন একটি শব্দ খুঁজতে বলতে পারি যার ভেক্টর উপস্থাপন *king* এবং *woman* শব্দের যতটা সম্ভব কাছাকাছি এবং *man* শব্দ থেকে যতটা সম্ভব দূরে:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

উপরের উদাহরণটি কিছু অভ্যন্তরীণ GenSym ম্যাজিক ব্যবহার করে, তবে এর অন্তর্নিহিত যুক্তি আসলে বেশ সহজ। এম্বেডিং সম্পর্কে একটি আকর্ষণীয় বিষয় হল যে আপনি এম্বেডিং ভেক্টরগুলিতে সাধারণ ভেক্টর অপারেশন করতে পারেন, এবং এটি শব্দের **অর্থ** নিয়ে অপারেশনগুলিকে প্রতিফলিত করবে। উপরের উদাহরণটি ভেক্টর অপারেশনের মাধ্যমে প্রকাশ করা যেতে পারে: আমরা **KING-MAN+WOMAN** এর সাথে সম্পর্কিত ভেক্টরটি গণনা করি (যেখানে সংশ্লিষ্ট শব্দগুলির ভেক্টর উপস্থাপনার উপর `+` এবং `-` অপারেশনগুলি সম্পাদিত হয়), এবং তারপর সেই ভেক্টরের সাথে অভিধানের সবচেয়ে কাছাকাছি শব্দটি খুঁজে বের করি:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: আমরা *man* এবং *woman* ভেক্টরগুলিতে একটি ছোট সহগ যোগ করতে হয়েছিল - এটি সরিয়ে দেখুন কী ঘটে।

নিকটতম ভেক্টর খুঁজে পেতে, আমরা TensorFlow এর মেশিনারি ব্যবহার করে আমাদের ভেক্টর এবং শব্দভাণ্ডারের সমস্ত ভেক্টরের মধ্যে দূরত্বের একটি ভেক্টর গণনা করি, এবং তারপর `argmin` ব্যবহার করে সর্বনিম্ন শব্দের সূচক খুঁজে বের করি।


যদিও Word2Vec শব্দের অর্থ বোঝানোর জন্য একটি চমৎকার পদ্ধতি বলে মনে হয়, এর বেশ কিছু সীমাবদ্ধতা রয়েছে, যার মধ্যে উল্লেখযোগ্য:

* CBoW এবং skip-gram উভয় মডেলই **predictive embeddings**, এবং এগুলো শুধুমাত্র স্থানীয় প্রসঙ্গ বিবেচনা করে। Word2Vec বৈশ্বিক প্রসঙ্গের সুবিধা গ্রহণ করে না।
* Word2Vec শব্দের **morphology** বিবেচনা করে না, অর্থাৎ শব্দের অর্থ তার বিভিন্ন অংশ, যেমন মূল শব্দের উপর নির্ভর করতে পারে—এই বিষয়টি উপেক্ষা করে।  

**FastText** দ্বিতীয় সীমাবদ্ধতাটি কাটিয়ে উঠতে চেষ্টা করে এবং Word2Vec-এর উপর ভিত্তি করে প্রতিটি শব্দ এবং প্রতিটি শব্দের মধ্যে থাকা চরিত্র n-grams-এর জন্য ভেক্টর উপস্থাপন শিখে। এই উপস্থাপনাগুলোর মান প্রতিটি প্রশিক্ষণ ধাপে একটি ভেক্টরে গড় করা হয়। যদিও এটি প্রাক-প্রশিক্ষণে অতিরিক্ত গণনার প্রয়োজন করে, এটি শব্দ embeddings-কে সাব-ওয়ার্ড তথ্য এনকোড করতে সক্ষম করে।

আরেকটি পদ্ধতি, **GloVe**, শব্দ embeddings-এর জন্য একটি ভিন্ন পদ্ধতি ব্যবহার করে, যা শব্দ-প্রসঙ্গ ম্যাট্রিক্সের ফ্যাক্টরাইজেশনের উপর ভিত্তি করে। প্রথমে এটি একটি বড় ম্যাট্রিক্স তৈরি করে, যা বিভিন্ন প্রসঙ্গে শব্দের উপস্থিতির সংখ্যা গণনা করে, এবং তারপর এটি এই ম্যাট্রিক্সকে নিম্ন মাত্রায় উপস্থাপন করার চেষ্টা করে, যাতে পুনর্গঠন ক্ষতি (reconstruction loss) ন্যূনতম হয়।

gensim লাইব্রেরি এই শব্দ embeddings সমর্থন করে, এবং আপনি উপরের মডেল লোডিং কোড পরিবর্তন করে এগুলোর সাথে পরীক্ষা-নিরীক্ষা করতে পারেন।


## কেরাসে প্রি-ট্রেইনড এমবেডিং ব্যবহার করা

উপরের উদাহরণটি আমরা পরিবর্তন করে আমাদের এমবেডিং লেয়ারে সেমান্টিক এমবেডিং, যেমন Word2Vec, দিয়ে ম্যাট্রিক্সটি প্রি-পপুলেট করতে পারি। প্রি-ট্রেইনড এমবেডিং এবং টেক্সট কর্পাসের ভোকাবুলারিগুলো সাধারণত একে অপরের সাথে মিলে না, তাই আমাদের একটি বেছে নিতে হবে। এখানে আমরা দুটি সম্ভাব্য বিকল্প নিয়ে আলোচনা করব: টোকেনাইজার ভোকাবুলারি ব্যবহার করা এবং Word2Vec এমবেডিংয়ের ভোকাবুলারি ব্যবহার করা।

### টোকেনাইজার ভোকাবুলারি ব্যবহার করা

যখন টোকেনাইজার ভোকাবুলারি ব্যবহার করা হয়, তখন ভোকাবুলারির কিছু শব্দের জন্য Word2Vec এমবেডিং পাওয়া যাবে, এবং কিছু শব্দ অনুপস্থিত থাকবে। আমাদের ভোকাবুলারির আকার যদি `vocab_size` হয় এবং Word2Vec এমবেডিং ভেক্টরের দৈর্ঘ্য যদি `embed_size` হয়, তাহলে এমবেডিং লেয়ারটি একটি ওজন ম্যাট্রিক্স দ্বারা উপস্থাপিত হবে যার আকৃতি হবে `vocab_size`$\times$`embed_size`। আমরা এই ম্যাট্রিক্সটি ভোকাবুলারির মধ্য দিয়ে গিয়ে পূরণ করব:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


শব্দগুলি যদি Word2Vec শব্দভাণ্ডারে উপস্থিত না থাকে, তাহলে আমরা সেগুলিকে শূন্য হিসাবে রাখতে পারি, অথবা একটি র্যান্ডম ভেক্টর তৈরি করতে পারি।

এখন আমরা প্রি-ট্রেইনড ওজন সহ একটি এমবেডিং লেয়ার সংজ্ঞায়িত করতে পারি:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **নোট**: লক্ষ্য করুন যে আমরা `trainable=False` সেট করেছি যখন `Embedding` তৈরি করা হয়েছে, যার অর্থ হলো আমরা Embedding লেয়ারটি পুনরায় প্রশিক্ষণ করছি না। এটি সঠিকতার হারকে সামান্য কমিয়ে দিতে পারে, তবে এটি প্রশিক্ষণের গতি বাড়ায়।

### এমবেডিং ভোকাবুলারি ব্যবহার করা

পূর্ববর্তী পদ্ধতির একটি সমস্যা হলো TextVectorization এবং Embedding-এ ব্যবহৃত ভোকাবুলারিগুলো ভিন্ন। এই সমস্যার সমাধান করতে, আমরা নিম্নলিখিত সমাধানগুলোর একটি ব্যবহার করতে পারি:
* আমাদের ভোকাবুলারির উপর Word2Vec মডেলটি পুনরায় প্রশিক্ষণ করা।
* প্রি-ট্রেইনড Word2Vec মডেলের ভোকাবুলারি ব্যবহার করে আমাদের ডেটাসেট লোড করা। ডেটাসেট লোড করার সময় ব্যবহৃত ভোকাবুলারি নির্দিষ্ট করা যেতে পারে।

দ্বিতীয় পদ্ধতিটি সহজ মনে হচ্ছে, তাই আমরা এটি বাস্তবায়ন করব। প্রথমেই, আমরা Word2Vec এমবেডিং থেকে নেওয়া নির্দিষ্ট ভোকাবুলারি দিয়ে একটি `TextVectorization` লেয়ার তৈরি করব:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

জেনসিম ওয়ার্ড এম্বেডিংস লাইব্রেরিতে একটি সুবিধাজনক ফাংশন, `get_keras_embeddings`, রয়েছে, যা আপনার জন্য স্বয়ংক্রিয়ভাবে সংশ্লিষ্ট কেরাস এম্বেডিংস লেয়ার তৈরি করবে।


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


আমাদের উচ্চতর সঠিকতা দেখতে না পাওয়ার একটি কারণ হল আমাদের ডেটাসেটের কিছু শব্দ প্রি-ট্রেইনড GloVe ভোকাবুলারিতে অনুপস্থিত, এবং তাই সেগুলি কার্যত উপেক্ষা করা হয়। এটি অতিক্রম করতে, আমরা আমাদের ডেটাসেটের উপর ভিত্তি করে আমাদের নিজস্ব এম্বেডিংস প্রশিক্ষণ করতে পারি।


## প্রসঙ্গগত এম্বেডিংস

প্রথাগত প্রি-ট্রেইনড এম্বেডিং প্রতিনিধিত্ব যেমন Word2Vec-এর একটি প্রধান সীমাবদ্ধতা হলো, যদিও এটি একটি শব্দের কিছু অর্থ ধারণ করতে পারে, এটি বিভিন্ন অর্থের মধ্যে পার্থক্য করতে পারে না। এটি ডাউনস্ট্রিম মডেলগুলিতে সমস্যা সৃষ্টি করতে পারে।

উদাহরণস্বরূপ, 'play' শব্দটির এই দুটি বাক্যে ভিন্ন অর্থ রয়েছে:
- আমি থিয়েটারে একটি **play** দেখতে গিয়েছিলাম।
- জন তার বন্ধুদের সাথে **play** করতে চায়।

আমরা যে প্রি-ট্রেইনড এম্বেডিংস নিয়ে আলোচনা করেছি, তা 'play' শব্দটির উভয় অর্থকে একই এম্বেডিংয়ে উপস্থাপন করে। এই সীমাবদ্ধতা কাটিয়ে উঠতে, আমাদের **ভাষা মডেল**-এর উপর ভিত্তি করে এম্বেডিং তৈরি করতে হবে, যা একটি বৃহৎ টেক্সট কর্পাসে প্রশিক্ষিত হয় এবং *জানে* কীভাবে শব্দগুলো বিভিন্ন প্রসঙ্গে একত্রিত হতে পারে। প্রসঙ্গগত এম্বেডিংস নিয়ে আলোচনা এই টিউটোরিয়ালের পরিধির বাইরে, তবে আমরা পরবর্তী ইউনিটে ভাষা মডেল নিয়ে আলোচনা করার সময় এ বিষয়ে ফিরে আসব।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
